In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image
import numpy as np
from sklearn import metrics

In [5]:
#adds all of the filepaths
train_path = "datasets\\cropped_lisa_1\\train_1"
test_path = "datasets\\cropped_lisa_1\\val_1"

In [6]:

train_data = []
test_data = []


for instance in os.listdir(train_path):
    for im in os.listdir(train_path + "\\" + instance):

        image = Image.open(train_path + "\\" + instance+ "\\" + im)
        image = np.array(image.resize((64,64)))
        image = list(image.reshape(64,64,3))
        
        if instance == "go": 
            train_data.append(image)
            test_data.append([1,0,0])
        elif instance == "warning" : 
            train_data.append(image)
            test_data.append([0,1,0])
        elif instance == "stop": 
            train_data.append(image)
            test_data.append([0,0,1])


In [7]:
train_data = np.array(train_data)
test_data = np.array(test_data)
print(train_data.shape, test_data.shape)

(28690, 64, 64, 3) (28690, 3)


In [8]:
#normalize to 0-1 for every pixel value
train_data = train_data/255
print(train_data.shape, test_data.shape)
print("NORMALIZED PIXEL VALUES:", train_data[0])


(28690, 64, 64, 3) (28690, 3)
NORMALIZED PIXEL VALUES: [[[0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  ...
  [0.14117647 0.20392157 0.20392157]
  [0.14901961 0.21176471 0.21176471]
  [0.14901961 0.21176471 0.21176471]]

 [[0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  ...
  [0.14117647 0.20392157 0.20392157]
  [0.14901961 0.21176471 0.21176471]
  [0.14901961 0.21176471 0.21176471]]

 [[0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  [0.08235294 0.07058824 0.15294118]
  ...
  [0.14117647 0.20392157 0.20392157]
  [0.14901961 0.21176471 0.21176471]
  [0.14901961 0.21176471 0.21176471]]

 ...

 [[0.05882353 0.06666667 0.05098039]
  [0.05882353 0.06666667 0.05098039]
  [0.05882353 0.06666667 0.05098039]
  ...
  [0.14509804 0.19607843 0.23137255]
  [0.15294118 0.20392157 0.23921569]
  [0.15686275 0.20784314 0.23921569]]

 [[0.05882353 0.066

In [9]:
np.random.seed(42)
np.random.shuffle(train_data)

#Use 80/20 split for training/testing
train_ration = .8
test_ratio = .2

total_samples = train_data.shape[0]
train_samples_count = int(.8 * total_samples)
val_samples_count = int(.2 * total_samples)

x_train = train_data[:train_samples_count]
x_val = train_data[train_samples_count:]

y_train = test_data[:train_samples_count]
y_val = test_data[train_samples_count:]

In [10]:
print(train_data.shape)
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

(28690, 64, 64, 3)
(22952, 64, 64, 3) (22952, 3)
(5738, 64, 64, 3) (5738, 3)


In [11]:
#Model creation
from tensorflow import keras


model = keras.Sequential([
    layers.Conv2D(32, (3,3), activation = "relu", input_shape = (64,64,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation = "relu"),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation = "relu"),
    layers.Dense(3, activation= "softmax")
])

model.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics = ['accuracy'])

In [12]:
#training the model

history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 5, batch_size = 16)

Epoch 1/5
2869/2869 [==============================] - 238s 82ms/step - loss: 0.6916 - accuracy: 0.5539 - val_loss: 2.2201 - val_accuracy: 0.0000e+00
Epoch 2/5
2869/2869 [==============================] - 209s 73ms/step - loss: 0.6870 - accuracy: 0.5594 - val_loss: 2.0791 - val_accuracy: 0.0000e+00
Epoch 3/5
2869/2869 [==============================] - 196s 68ms/step - loss: 0.6867 - accuracy: 0.5594 - val_loss: 2.2147 - val_accuracy: 0.0000e+00
Epoch 4/5
2869/2869 [==============================] - 192s 67ms/step - loss: 0.6866 - accuracy: 0.5594 - val_loss: 2.3793 - val_accuracy: 0.0000e+00
Epoch 5/5
2869/2869 [==============================] - 239s 83ms/step - loss: 0.6864 - accuracy: 0.5594 - val_loss: 2.6833 - val_accuracy: 0.0000e+00


In [21]:
validation_final_x = []
validation_final_y = []

test_size = 1000

for instance in os.listdir(test_path):
    counter = 0
    for im in os.listdir(test_path + "\\" + instance):
        counter+=1
        if counter%(test_size//3) == 0:
            break
        image = Image.open(test_path + "\\" + instance+ "\\" + im)
        image = np.array(image.resize((64,64)))
        image = list(image.reshape(64,64,3))
        
        if instance == "go": 
            validation_final_x.append(image)
            validation_final_y.append(0)
        elif instance == "warning" : 
            validation_final_x.append(image)
            validation_final_y.append(1)
        elif instance == "stop": 
            validation_final_x.append(image)
            validation_final_y.append(2)

In [22]:
#count of final test 
validation_final_x = np.array(validation_final_x)
validation_final_y = np.array(validation_final_y)
print(validation_final_x.shape, validation_final_y.shape)

(772, 64, 64, 3) (772,)


In [24]:
#finding accuracy of final testing
from sklearn.metrics import accuracy_score
predictions = model.predict(validation_final_x)
predicted_labels = np.argmax(predictions, axis = 1)
print(predicted_labels.shape)
accuracy = accuracy_score(predicted_labels, validation_final_y)

25/25 [==============================] - 0s 17ms/step
(772,)


In [25]:
print(f"Final Accuracy: {accuracy}")

Accuracy: 0.4961139896373057
